In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

# Data

In [3]:
from kegra.utils import load_data, preprocess_adj, get_splits

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 14:34:26.142214 140009850488640 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 14:34:26.149286 140009850488640 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 14:34:26.154808 140009850488640 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import evaluate_preds

In [11]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 14:34:26.165310 140009850488640 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 14:34:26.170295 140009850488640 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 14:34:26.179300 140009850488640 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [12]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 14:34:26.220656 140009850488640 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 14:34:26.290377 140009850488640 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9334 train_acc= 0.4357 val_loss= 1.9348 val_acc= 0.4033 time= 1.1041
Epoch: 0002 train_loss= 1.9210 train_acc= 0.4214 val_loss= 1.9239 val_acc= 0.4033 time= 0.0235
Epoch: 0003 train_loss= 1.9068 train_acc= 0.4643 val_loss= 1.9121 val_acc= 0.4200 time= 0.0268
Epoch: 0004 train_loss= 1.8922 train_acc= 0.4857 val_loss= 1.9005 val_acc= 0.4633 time= 0.0184
Epoch: 0005 train_loss= 1.8779 train_acc= 0.4857 val_loss= 1.8891 val_acc= 0.4633 time= 0.0194
Epoch: 0006 train_loss= 1.8630 train_acc= 0.4857 val_loss= 1.8769 val_acc= 0.4633 time= 0.0187
Epoch: 0007 train_loss= 1.8477 train_acc= 0.4857 val_loss= 1.8642 val_acc= 0.4800 time= 0.0235
Epoch: 0008 train_loss= 1.8321 train_acc= 0.4786 val_loss= 1.8514 val_acc= 0.4800 time= 0.0198
Epoch: 0009 train_loss= 1.8169 train_acc= 0.4786 val_loss= 1.8388 val_acc= 0.4767 time= 0.0194
Epoch: 0010 train_loss= 1.8021 train_acc= 0.4786 val_loss= 1.8268 val_acc= 0.4733 time= 0.0238
Epoch: 0011 train_loss= 1.7872 train_acc= 0.4786 v

Epoch: 0096 train_loss= 0.9579 train_acc= 0.8071 val_loss= 1.2083 val_acc= 0.7133 time= 0.0222
Epoch: 0097 train_loss= 0.9505 train_acc= 0.8143 val_loss= 1.2026 val_acc= 0.7200 time= 0.0238
Epoch: 0098 train_loss= 0.9432 train_acc= 0.8357 val_loss= 1.1968 val_acc= 0.7233 time= 0.0183
Epoch: 0099 train_loss= 0.9360 train_acc= 0.8429 val_loss= 1.1911 val_acc= 0.7233 time= 0.0224
Epoch: 0100 train_loss= 0.9287 train_acc= 0.8500 val_loss= 1.1854 val_acc= 0.7233 time= 0.0223
Epoch: 0101 train_loss= 0.9214 train_acc= 0.8500 val_loss= 1.1797 val_acc= 0.7233 time= 0.0199
Epoch: 0102 train_loss= 0.9141 train_acc= 0.8500 val_loss= 1.1740 val_acc= 0.7233 time= 0.0222
Epoch: 0103 train_loss= 0.9072 train_acc= 0.8500 val_loss= 1.1681 val_acc= 0.7267 time= 0.0226
Epoch: 0104 train_loss= 0.9006 train_acc= 0.8500 val_loss= 1.1620 val_acc= 0.7267 time= 0.0182
Epoch: 0105 train_loss= 0.8944 train_acc= 0.8571 val_loss= 1.1565 val_acc= 0.7300 time= 0.0181
Epoch: 0106 train_loss= 0.8880 train_acc= 0.8643 v

Epoch: 0185 train_loss= 0.5563 train_acc= 0.9429 val_loss= 0.8949 val_acc= 0.7967 time= 0.0227
Epoch: 0186 train_loss= 0.5548 train_acc= 0.9429 val_loss= 0.8935 val_acc= 0.7900 time= 0.0233
Epoch: 0187 train_loss= 0.5531 train_acc= 0.9429 val_loss= 0.8915 val_acc= 0.7900 time= 0.0184
Epoch: 0188 train_loss= 0.5504 train_acc= 0.9429 val_loss= 0.8898 val_acc= 0.7967 time= 0.0206
Epoch: 0189 train_loss= 0.5478 train_acc= 0.9357 val_loss= 0.8888 val_acc= 0.7967 time= 0.0232
Epoch: 0190 train_loss= 0.5457 train_acc= 0.9357 val_loss= 0.8877 val_acc= 0.7967 time= 0.0240
Epoch: 0191 train_loss= 0.5439 train_acc= 0.9286 val_loss= 0.8879 val_acc= 0.7933 time= 0.0187
Epoch: 0192 train_loss= 0.5422 train_acc= 0.9214 val_loss= 0.8885 val_acc= 0.8000 time= 0.0182
Epoch: 0193 train_loss= 0.5397 train_acc= 0.9214 val_loss= 0.8884 val_acc= 0.7967 time= 0.0186
Epoch: 0194 train_loss= 0.5364 train_acc= 0.9286 val_loss= 0.8870 val_acc= 0.8000 time= 0.0217
Epoch: 0195 train_loss= 0.5325 train_acc= 0.9286 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.9388
accuracy = 0.7650


# Gaussian Graph Convolution

In [15]:
from ggcn import GaussianGraphConvolution

In [16]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GaussianGraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

In [17]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [18]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9327 train_acc= 0.3571 val_loss= 1.9338 val_acc= 0.3700 time= 0.4164
Epoch: 0002 train_loss= 1.9192 train_acc= 0.3643 val_loss= 1.9217 val_acc= 0.3633 time= 0.0231
Epoch: 0003 train_loss= 1.9051 train_acc= 0.3857 val_loss= 1.9096 val_acc= 0.3667 time= 0.0192
Epoch: 0004 train_loss= 1.8907 train_acc= 0.3929 val_loss= 1.8974 val_acc= 0.3733 time= 0.0228
Epoch: 0005 train_loss= 1.8762 train_acc= 0.4000 val_loss= 1.8851 val_acc= 0.3767 time= 0.0196
Epoch: 0006 train_loss= 1.8614 train_acc= 0.4071 val_loss= 1.8726 val_acc= 0.3767 time= 0.0233
Epoch: 0007 train_loss= 1.8465 train_acc= 0.4071 val_loss= 1.8599 val_acc= 0.3767 time= 0.0281
Epoch: 0008 train_loss= 1.8313 train_acc= 0.4071 val_loss= 1.8470 val_acc= 0.3733 time= 0.0224
Epoch: 0009 train_loss= 1.8161 train_acc= 0.4071 val_loss= 1.8339 val_acc= 0.3733 time= 0.0186
Epoch: 0010 train_loss= 1.8013 train_acc= 0.4000 val_loss= 1.8215 val_acc= 0.3733 time= 0.0300
Epoch: 0011 train_loss= 1.7868 train_acc= 0.4000 v

Epoch: 0089 train_loss= 0.8813 train_acc= 0.8286 val_loss= 1.1388 val_acc= 0.7433 time= 0.0203
Epoch: 0090 train_loss= 0.8740 train_acc= 0.8429 val_loss= 1.1313 val_acc= 0.7433 time= 0.0259
Epoch: 0091 train_loss= 0.8669 train_acc= 0.8429 val_loss= 1.1242 val_acc= 0.7433 time= 0.0208
Epoch: 0092 train_loss= 0.8598 train_acc= 0.8429 val_loss= 1.1176 val_acc= 0.7467 time= 0.0236
Epoch: 0093 train_loss= 0.8527 train_acc= 0.8429 val_loss= 1.1115 val_acc= 0.7467 time= 0.0241
Epoch: 0094 train_loss= 0.8453 train_acc= 0.8500 val_loss= 1.1055 val_acc= 0.7533 time= 0.0206
Epoch: 0095 train_loss= 0.8378 train_acc= 0.8500 val_loss= 1.1003 val_acc= 0.7433 time= 0.0194
Epoch: 0096 train_loss= 0.8303 train_acc= 0.8571 val_loss= 1.0960 val_acc= 0.7533 time= 0.0190
Epoch: 0097 train_loss= 0.8232 train_acc= 0.8571 val_loss= 1.0919 val_acc= 0.7600 time= 0.0214
Epoch: 0098 train_loss= 0.8161 train_acc= 0.8571 val_loss= 1.0868 val_acc= 0.7633 time= 0.0210
Epoch: 0099 train_loss= 0.8090 train_acc= 0.8643 v

Epoch: 0179 train_loss= 0.5038 train_acc= 0.9714 val_loss= 0.8325 val_acc= 0.8200 time= 0.0202
Epoch: 0180 train_loss= 0.5013 train_acc= 0.9714 val_loss= 0.8301 val_acc= 0.8233 time= 0.0246
Epoch: 0181 train_loss= 0.4981 train_acc= 0.9714 val_loss= 0.8285 val_acc= 0.8233 time= 0.0183
Epoch: 0182 train_loss= 0.4948 train_acc= 0.9643 val_loss= 0.8276 val_acc= 0.8233 time= 0.0195
Epoch: 0183 train_loss= 0.4917 train_acc= 0.9643 val_loss= 0.8271 val_acc= 0.8200 time= 0.0238
Epoch: 0184 train_loss= 0.4888 train_acc= 0.9643 val_loss= 0.8260 val_acc= 0.8100 time= 0.0201
Epoch: 0185 train_loss= 0.4855 train_acc= 0.9643 val_loss= 0.8219 val_acc= 0.8133 time= 0.0229
Epoch: 0186 train_loss= 0.4826 train_acc= 0.9643 val_loss= 0.8182 val_acc= 0.8133 time= 0.0192
Epoch: 0187 train_loss= 0.4798 train_acc= 0.9643 val_loss= 0.8165 val_acc= 0.8067 time= 0.0188
Epoch: 0188 train_loss= 0.4772 train_acc= 0.9643 val_loss= 0.8158 val_acc= 0.8233 time= 0.0185
Epoch: 0189 train_loss= 0.4755 train_acc= 0.9643 v

In [19]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8679
accuracy = 0.7920


# Attacks